# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770904698735                   -0.53    9.0    199ms
  2   -2.772147228434       -2.91       -1.32    1.0    106ms
  3   -2.772170496369       -4.63       -2.52    1.0    468ms
  4   -2.772170679481       -6.74       -3.33    1.0   98.9ms
  5   -2.772170721367       -7.38       -3.90    2.0    145ms
  6   -2.772170722982       -8.79       -5.29    1.0    137ms
  7   -2.772170723013      -10.51       -5.34    2.0    121ms
  8   -2.772170723015      -11.70       -6.12    1.0    107ms
  9   -2.772170723015      -13.19       -6.45    2.0    120ms
 10   -2.772170723015      -13.59       -7.55    1.0    108ms
 11   -2.772170723015   +  -14.40       -7.90    1.0    109ms
 12   -2.772170723015   +    -Inf       -8.73    1.0    108ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.773557944571622

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770682786334                   -0.53    8.0    159ms
  2   -2.772051318557       -2.86       -1.31    1.0    141ms
  3   -2.772082881078       -4.50       -2.60    1.0    461ms
  4   -2.772083384372       -6.30       -3.73    1.0   99.0ms
  5   -2.772083417302       -7.48       -4.18    2.0    115ms
  6   -2.772083417780       -9.32       -5.26    1.0    104ms
  7   -2.772083417809      -10.54       -5.46    2.0    119ms
  8   -2.772083417810      -11.93       -6.21    1.0    110ms
  9   -2.772083417811      -12.90       -6.59    2.0    125ms
 10   -2.772083417811   +  -14.75       -7.35    1.0    112ms
 11   -2.772083417811      -14.12       -7.94    1.0    113ms
 12   -2.772083417811   +  -14.18       -8.46    1.0    111ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ----